<center>
    <img src="http://sct.inf.utfsm.cl/wp-content/uploads/2020/04/logo_di.png" style="width:60%">
    <h1> INF285/ILI285 Computación Científica </h1>
    <h1> COP-R</h1>
    <h1> Representación Punto Flotante</h1>
</center>

In [1]:
import numpy as np
import bitstring as bs

La nueva compañía de buses "Viajes Distanciados" ha armado una flota de buses donde en su interior tienen sensores que permiten monitorear a sus pasajeros para fiscalizar que siguen las normas de seguridad recomendadas por la OMS. Las mediciones de estos sensores se resumen en un único coeficiente y estos son desplegados en un tablero como el siguiente:

|          |    B1   |    B2    |     B3    |     B4    |    B5    |
|:--------:|:-------:|:--------:|:---------:|:---------:|:--------:|
| Sensor 1 |   $25$  |  $-300$  |   $1050$  |  $-1400$  |   $630$  |
| Sensor 2 |  $-300$ |  $4800$  |  $-18900$ |  $26880$  | $-12600$ |
| Sensor 3 |  $1050$ | $-18900$ |  $79380$  | $-117600$ |  $56700$ |
| Sensor 4 | $-1400$ |  $26880$ | $-117600$ |  $179200$ | $-88200$ |
| Sensor 5 |  $630$  | $-12600$ |  $56700$  |  $-88200$ |  $44100$ |

donde  B1 a B5 son los id de los buses.

Se le pide desarrollar una representación punto flotante que permita almacenar el menor y mayor número mostrado en la tabla anterior y además se debe cumplir con las siguientes restricciones:

La primera restricción consiste en poder almacenar la solución $\mathbf{x}$ del sistema de ecuaciones lineales $A\,\mathbf{x} = \mathbf{1}$ obtenida usando doble precisión, siendo $A$ los valores del tablero y $\mathbf{1}$ el vector de $1$s, tal que al obtener la norma $2$ del residuo se obtenga un valor menor a $1$. Por ejemplo, supongámonos que tenemos la siguiente tabla:

|          |      B1      |      B2      |
|:--------:|:------------:|:------------:|
| Sensor 1 | $0.12339028$ |  $0.0559545$ |
| Sensor 2 | $0.05606236$ | $0.13714786$ |
 

Resolviendo el sistema $$A\,\mathbf{x} = \mathbf{1}$$ obtenemos que la solución es el vector $\mathbf{x} = [5.88964611, 4.88387183]$. Si suponemos que la representación utiliza dos bits de mantisa entonces el vector $\mathbf{x}$ almacenado sería igual a $[5,4]$ debido a que $1.01 \times 2^2 = (1 \times 2^2) + (0 \times 2^1) + (1 \times 2^0) = 5$ y $1.00 \times 2^2 = (1 \times 2^2) + (0 \times 2^1) + (0 \times 2^0) = 4.$ Si calculamos la norma del residuo en doble precisión obtendremos el valor $0.23$, como dio menor a $1$ entonces utilizar dos bits de mantisa cumpliría con lo que se busca.

La segunda restricción consiste en utilizar la menor cantidad posible de bits para el exponente, mantisa y signo. Incluso ocupando menos bits totales que una representación IEEE single precisión.
Una vez encontrada la representación, conteste las siguiente preguntas. (Para escribir potencias en las respuestas puedes utilizar lo siguiente: 2E-4)

* [7 puntos] ¿Cuántos bits se necesitan para el signo? Para el signo es necesario un bit.
* [7 puntos] ¿Cuántos bits se necesitan para la mantisa? Se necesitaran al menos $17$ bits en la mantisa. 
* [12 puntos] ¿Cuántos bits se necesitan para el exponente? Con $6$ bits es suficiente para poder representar esos números, ya que con $5$ el máximo tendremos números del orden de $32768$ y eso no es suficiente para $179200$.
* [7 puntos] ¿Cual es el $e_{mach}$ de la representación? $2^{-17}$
* [7 puntos] ¿Cuál es el menor número representable mayor que $0$? En representación subnormal

In [2]:
def to_binary(f):
    b = bs.pack('>d', f)
    b = b.bin
    #show sign + exponent + mantisa
    return(b[0]+' '+b[1:12]+ ' '+b[12:])

In [3]:
## Mantisa

A = np.array([[25, -300, 1050, -1400, 630],
             [-300, 4800, -18900, 26880,-12600],
             [1050, -18900, 79380, -117600, 56700],
             [-1400, 26880, -117600, 179200, -88200],
             [630, - 12600, 56700, -88200, 44100]])
b = np.ones(A.shape[0])

a = np.linalg.solve(A,b)
for i in range(2,30):
    vector_nuevo = []
    for ff in a:
        t = list(to_binary(ff).split(" ")[2])
        resultado2 = 0
        count2 = 10
        for rr in list(to_binary(ff).split(" ")[1]):
            resultado2 += int(rr)*2**count2
            count2-=1
        resultado2 -= 1023
        k = i
        count = -1
        resultado = 1*2**resultado2
        for h in t[:k]:
            resultado += (int(h)*2**count)*2**resultado2
            count-=1

        vector_nuevo.append(resultado)

    print("b aproximado:", np.dot(A,np.array(vector_nuevo)))
    print("Norma del vector residuo:", np.linalg.norm(b-np.dot(A,np.array(vector_nuevo))))
    print("------")
    if np.linalg.norm(b-np.dot(A,np.array(vector_nuevo))) < 1:
              break
print(i)

b aproximado: [ -106.25  2145.   -9607.5  14875.   -7402.5 ]
Norma del vector residuo: 19312.669392461
------
b aproximado: [  -98.125  1882.5   -8163.75  12372.5   -6063.75 ]
Norma del vector residuo: 16126.010523084282
------
b aproximado: [  -31.5625   607.5    -2611.875   3946.25   -1929.375 ]
Norma del vector residuo: 5146.587562177122
------
b aproximado: [  -21.71875   410.625   -1725.9375   2568.125   -1240.3125 ]
Norma del vector residuo: 3358.8107120547984
------
b aproximado: [ -10.546875  217.5      -928.59375  1397.8125   -679.21875 ]
Norma del vector residuo: 1823.4398340414582
------
b aproximado: [  -4.6875   107.34375 -456.09375  689.0625  -334.6875 ]
Norma del vector residuo: 897.9933956727493
------
b aproximado: [  -0.52734375   27.65625    -109.1015625   162.421875    -76.2890625 ]
Norma del vector residuo: 211.8156354440793
------
b aproximado: [  0.77148438   4.6875     -13.53515625  21.6015625   -8.61328125]
Norma del vector residuo: 27.235274859743768
------
b 

In [4]:
## Exponente
n = 6
a = ''
for k in range(0,n):
    a+='1'
r = int(a,2) 
shift = 2**(n-1) - 1
print("Intervalo",1-shift, r-1-shift)
print(2**(r-1-shift))

Intervalo -30 31
2147483648
